In [1]:
#类的方式
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name, fidelity')

class lineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity
    
class Order:
    
    def __init__(self, customer, cart, promotion = None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in cart)
        return self.__total
    
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())
    
#定义抽象基类
class Promotion(ABC):
    @abstractmethod
    def discount(self, order):
        """打折抽象类"""

class fidelityPromo(Promotion):
    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0
    
class bulkItemPromo(Promotion):
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.1
        return discount
    
class largeOrderPromo(Promotion):
    def discount(self, order):
        items = {item.product for item in order.cart}
        if len(items) > 0:
            return order.total() * 0.07
        return 0

In [2]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [
    lineItem('banana', 4, 5),
    lineItem('apple', 10, 1.5),
    lineItem('watermellon', 20, 5)
]
print(Order(joe, cart, fidelityPromo()))
print(Order(ann, cart, fidelityPromo()))

<Order total: 135.00 due: 135.00>
<Order total: 135.00 due: 128.25>


In [3]:
#函数实现 在函数是一等对象的情况下 这种方式会好很多，不用每次都创建一个实例算法，虽说可以用单例解决
from collections import namedtuple

Customer = namedtuple('Customer', 'name, fidelity')

class lineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity
    
class Order:
    
    def __init__(self, customer, cart, promotion = None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in cart)
        return self.__total
    
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())
    

def fidelityPromo(order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0
    
def bulkItemPromo(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount
    
def largeOrderPromo(order):
    items = {item.product for item in order.cart}
    if len(items) > 0:
        return order.total() * 0.07
    return 0

In [4]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [
    lineItem('banana', 4, 5),
    lineItem('apple', 10, 1.5),
    lineItem('watermellon', 20, 5)
]
print(Order(joe, cart, fidelityPromo))
print(Order(ann, cart, fidelityPromo))

<Order total: 135.00 due: 135.00>
<Order total: 135.00 due: 128.25>


In [5]:
#自动选择最好的算法 利用反射(python的内省)
promos = [globals()[name] for name in globals()
    if name.endswith('Promo')
    and name != 'bestPromo'
]

def bestPromo(order):
    return max(promo(order) for promo in promos)

print(Order(joe, cart, bestPromo))
print(Order(ann, cart, bestPromo))

<Order total: 135.00 due: 125.00>
<Order total: 135.00 due: 125.00>


In [13]:
#利用装饰器进行选择
#函数实现 在函数是一等对象的情况下 这种方式会好很多，不用每次都创建一个实例算法，虽说可以用单例解决
from collections import namedtuple
import functools

Customer = namedtuple('Customer', 'name, fidelity')

class lineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity
    
class Order:
    
    def __init__(self, customer, cart, promotion = None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in cart)
        return self.__total
    
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

promos = []
def promotion(promoFunc):
    promos.append(promoFunc)
    return promoFunc

@promotion
def fidelityPromo(order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulkItemPromo(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

@promotion
def largeOrderPromo(order):
    items = {item.product for item in order.cart}
    if len(items) > 0:
        return order.total() * 0.07
    return 0

def bestPromo(order):
    return max(promo(order) for promo in promos)

In [14]:
print(Order(joe, cart, bestPromo))
print(Order(ann, cart, bestPromo))

<Order total: 135.00 due: 125.00>
<Order total: 135.00 due: 125.00>


In [12]:
fidelityPromo.__code__.co_freevars

()